In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegressionCV,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import r2_score,f1_score,accuracy_score,recall_score,precision_score,mean_absolute_error,mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from statsmodels.api import OLS
from xgboost import XGBClassifier,XGBRegressor

# Load DataSets

In [153]:
train = pd.read_csv('train_users_2.csv')
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [154]:
age_gender_data = pd.read_csv('age_gender_bkts.csv')
age_gender_data.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


In [156]:
countries = pd.read_csv('countries.csv')
countries

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


In [125]:
test = pd.read_csv('test_users.csv')

In [171]:
submission = pd.read_csv('sample_submission_NDF.csv')


In [172]:
sessions = pd.read_csv('sessions.csv')
pd.set_option('display.max_columns', None)
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


# Checking for Null Values

In [173]:
sessions.isnull().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [174]:
sessions.shape

(10567737, 6)

In [175]:
sessions.fillna(method='bfill',inplace=True)
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,click,view_search_results,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,click,view_search_results,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,click,view_search_results,Windows Desktop,435.0


In [176]:
sessions = sessions.groupby('user_id').agg(lambda x : x.tolist())
sessions

,action,action_type,action_detail,device_type,secs_elapsed
user_id,,,,,
00023iyk9l,"[index, dashboard, header_userpic, dashboard, ...","[view, view, data, view, partner_callback, mes...","[view_search_results, dashboard, header_userpi...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[20438.0, 787.0, 850.0, 934.0, 129817.0, 12981..."
0010k6l0om,"[search_results, show, personalize, show, sear...","[click, view, data, click, click, click, data,...","[view_search_results, p3, wishlist_content_upd...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[1708.0, 21260.0, 1223.0, 26.0, 847.0, 1230.0,..."
001wyh0pz8,"[search, search, search, show, social_connecti...","[click, click, click, view, data, -unknown-, v...","[view_search_results, view_search_results, vie...","[Android App Unknown Phone/Tablet, Android App...","[622.0, 1813.0, 1507.0, 6327.0, 927.0, 142.0, ..."
0028jgx1x1,"[show, reviews, show, search, show, search, re...","[view, data, view, click, view, click, data, s...","[user_profile, listing_reviews, p3, view_searc...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[6162.0, 75.0, 86.0, 13710.0, 25217.0, 10989.0..."
002qnbzfs5,"[social_connections, payment_methods, create, ...","[data, -unknown-, -unknown-, view, data, data,...","[user_social_connections, -unknown-, -unknown-...","[iPhone, iPhone, iPhone, iPhone, iPhone, iPhon...","[17135.0, 711.0, 274.0, 179.0, 483.0, 1.0, 782..."
...,...,...,...,...,...
zzxox7jnrx,"[identity, kba, kba_update, kba_update, popula...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[Windows Desktop, Windows Desktop, Windows Des...","[1338.0, 10115.0, 23802.0, 16951.0, 49938.0, 7..."
zzy7t0y9cm,"[personalize, header_userpic, create, personal...","[data, data, submit, data, view, click, click,...","[wishlist_content_update, header_userpic, crea...","[Windows Desktop, Windows Desktop, Windows Des...","[501.0, 3671.0, 42612.0, 42612.0, 697.0, 25616..."
zzysuoqg6x,"[hosting_social_proof, create, header_userpic]","[-unknown-, submit, data]","[-unknown-, create_user, header_userpic]","[Windows Desktop, Windows Desktop, Windows Des...","[1533.0, 198.0, 198.0]"


In [177]:
sessions['action'] = sessions['action'].apply(lambda x : ','.join(map(str,x)))

In [178]:
sessions['action_type'] = sessions['action_type'].apply(lambda x : ','.join(map(str,x)))

In [179]:
sessions['action_detail'] = sessions['action_detail'].apply(lambda x : ','.join(map(str,x)))

In [180]:
sessions['device_type'] = sessions['device_type'].apply(lambda x : ','.join(map(str,x)))

In [ ]:
joint_dt = pd.merge(train,sessions,how='inner',left_on='id',right_on='user_id')
pd.set_option('display.max_columns', None)
joint_dt

In [181]:
sessions['secs_elapsed'] = sessions['secs_elapsed'].apply(lambda x : np.sum(x))

In [182]:
sessions

,action,action_type,action_detail,device_type,secs_elapsed
user_id,,,,,
00023iyk9l,"index,dashboard,header_userpic,dashboard,callb...","view,view,data,view,partner_callback,message_p...","view_search_results,dashboard,header_userpic,d...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",997713.0
0010k6l0om,"search_results,show,personalize,show,search_re...","click,view,data,click,click,click,data,data,vi...","view_search_results,p3,wishlist_content_update...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",586565.0
001wyh0pz8,"search,search,search,show,social_connections,i...","click,click,click,view,data,-unknown-,view,-un...","view_search_results,view_search_results,view_s...","Android App Unknown Phone/Tablet,Android App U...",284468.0
0028jgx1x1,"show,reviews,show,search,show,search,reviews,c...","view,data,view,click,view,click,data,submit,-u...","user_profile,listing_reviews,p3,view_search_re...","-unknown-,-unknown-,-unknown-,-unknown-,-unkno...",297499.0
002qnbzfs5,"social_connections,payment_methods,create,show...","data,-unknown-,-unknown-,view,data,data,data,d...","user_social_connections,-unknown-,-unknown-,us...","iPhone,iPhone,iPhone,iPhone,iPhone,iPhone,iPho...",6487092.0
...,...,...,...,...,...
zzxox7jnrx,"identity,kba,kba_update,kba_update,populate_fr...","-unknown-,-unknown-,-unknown-,-unknown-,-unkno...","-unknown-,-unknown-,-unknown-,-unknown-,-unkno...","Windows Desktop,Windows Desktop,Windows Deskto...",643788.0
zzy7t0y9cm,"personalize,header_userpic,create,personalize,...","data,data,submit,data,view,click,click,submit","wishlist_content_update,header_userpic,create_...","Windows Desktop,Windows Desktop,Windows Deskto...",116383.0
zzysuoqg6x,"hosting_social_proof,create,header_userpic","-unknown-,submit,data","-unknown-,create_user,header_userpic","Windows Desktop,Windows Desktop,Windows Desktop",1929.0


In [184]:
train_join = pd.merge(train , sessions , how='inner' , left_on='id', right_on='user_id')
train_join

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other,"lookup,search_results,lookup,search_results,lo...","click,click,click,click,click,click,data,data,...","view_search_results,view_search_results,view_s...","Windows Desktop,Windows Desktop,Windows Deskto...",3427541.0
1,yo8nz8bqcq,2014-01-01,20140101001558,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,"dashboard,create,confirm_email,show,show_perso...","view,submit,click,view,data,view,data,data,-un...","dashboard,create_user,confirm_email_link,p3,us...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",323825.0
2,4grx6yxeby,2014-01-01,20140101001639,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Firefox,NDF,"verify,create,pending,pending,requested,header...","-unknown-,submit,message_post,booking_request,...","-unknown-,create_user,message_post,pending,p5,...","Windows Desktop,Windows Desktop,Windows Deskto...",1195245.0
3,ncf87guaf0,2014-01-01,20140101002146,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,"lookup,show,search_results,search_results,show...","view,view,click,click,view,view,click,data,dat...","p3,p3,view_search_results,view_search_results,...","Windows Desktop,Windows Desktop,Windows Deskto...",3755465.0
4,4rvqpxoh3h,2014-01-01,20140101002619,2014-01-02,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,GB,"campaigns,active,create,notifications,listings...","-unknown-,-unknown-,-unknown-,-unknown-,-unkno...","-unknown-,-unknown-,-unknown-,-unknown-,-unkno...","iPhone,iPhone,iPhone,iPhone,iPhone,iPhone,iPho...",2742.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73810,zxodksqpep,2014-06-30,20140630235636,NaN,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF,"edit,edit,handle_vanity_url,edit,references,re...","view,view,-unknown-,view,view,view,-unknown-,-...","edit_profile,edit_profile,-unknown-,edit_profi...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",5157153.0
73811,mhewnxesx9,2014-06-30,20140630235719,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,"confirm_email,dashboard,header_userpic,dashboa...","click,view,data,view,submit,view,view,data,vie...","confirm_email_link,dashboard,header_userpic,da...","iPhone,iPhone,iPhone,iPhone,Windows Desktop,Wi...",2878989.0
73812,6o3arsjbb4,2014-06-30,20140630235754,NaN,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,"ajax_refresh_subtotal,ajax_refresh_subtotal,si...","click,click,data,data,data,data,view,view,view...","change_trip_characteristics,change_trip_charac...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",343032.0
73813,jh95kwisub,2014-06-30,20140630235822,NaN,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF,"search,search,search,search,search,search,camp...","click,click,click,click,click,click,-unknown-,...","view_search_results,view_search_results,view_s...","iPhone,iPhone,iPhone,iPhone,iPhone,iPhone,iPho...",350398.0


In [185]:
train_join.isnull().sum()

id                             0
date_account_created           0
timestamp_first_active         0
date_first_booking         45041
gender                         0
age                        32248
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel              0
affiliate_provider             0
first_affiliate_tracked      302
signup_app                     0
first_device_type              0
first_browser                  0
country_destination            0
action                         0
action_type                    0
action_detail                  0
device_type                    0
secs_elapsed                   0
dtype: int64

In [192]:
test_join = pd.merge(test, sessions , how='left' , left_on='id', right_on='user_id')
test_join

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,action,action_type,action_detail,device_type,secs_elapsed
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,"show,search,search,show,authenticate,reviews,a...","view,click,click,view,submit,data,-unknown-,click","user_profile,view_search_results,view_search_r...","-unknown-,-unknown-,-unknown-,-unknown-,iPhone...",120284.0
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,"dashboard,login,search,create,search,signup_lo...","view,view,click,submit,click,view,-unknown-,vi...","dashboard,login_page,view_search_results,creat...","iPhone,iPhone,-unknown-,iPhone,-unknown-,iPhon...",253075.0
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,"index,index,index,search_results,search_result...","view,view,view,click,click,click,click,data,vi...","view_search_results,view_search_results,view_s...","Windows Desktop,Windows Desktop,Windows Deskto...",976343.0
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,"personalize,header_userpic,header_userpic,show...","data,data,data,view,view,data,data,click,submi...","wishlist_content_update,header_userpic,header_...","Windows Desktop,Windows Desktop,Windows Deskto...",136415.0
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,"message_to_host_change,agree_terms_check,pendi...","click,-unknown-,booking_request,message_post,v...","message_to_host_change,-unknown-,pending,messa...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",454125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE,"confirm_email,authenticate,index,header_userpi...","click,submit,view,data,submit,submit,data,data...","confirm_email_link,login,your_listings,header_...","iPhone,iPhone,iPhone,iPhone,Windows Desktop,Wi...",2534531.0
62092,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,NaN,basic,23,ko,direct,direct,untracked,Android,Android Phone,-unknown-,"index,campaigns,collections,active,show,show,s...","view,view,view,-unknown-,view,view,view,view,v...","view_reservations,user_wishlists,user_wishlist...","Android Phone,Android Phone,Android Phone,Andr...",58315.0
62093,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,NaN,basic,0,de,direct,direct,linked,Web,Windows Desktop,Firefox,"index,my,index,search_results,search_results,s...","view,view,view,click,click,click,click,click,c...","view_search_results,user_wishlists,user_wishli...","Windows Desktop,Windows Desktop,Windows Deskto...",825646.0
62094,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,"show,update,create,personalize,header_userpic,...","submit,submit,-unknown-,data,data,submit,view,...","update_listing,update_listing,-unknown-,wishli...","Mac Desktop,Mac Desktop,Mac Desktop,Mac Deskto...",373130.0


In [193]:
test_join.isnull().sum()

id                             0
date_account_created           0
timestamp_first_active         0
date_first_booking         62096
gender                         0
age                        28876
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel              0
affiliate_provider             0
first_affiliate_tracked       20
signup_app                     0
first_device_type              0
first_browser                  0
action                       428
action_type                  428
action_detail                428
device_type                  428
secs_elapsed                 428
dtype: int64

In [194]:
sessions_df = pd.read_csv('sessions.csv')

In [195]:
session1 = sessions_df.groupby(["user_id","action"]).agg({"action": "count"})
session2 = sessions_df.groupby(["user_id","action_type"]).agg({"action_type": "count"})
session3 = sessions_df.groupby(["user_id","action_detail"]).agg({"action_detail": "count"}) 

In [196]:
session1

action
user_id    action                       
00023iyk9l ajax_refresh_subtotal       2
           callback                    1
           confirm_email               1
           dashboard                   4
           header_userpic              2
...                                  ...
zzzlylp57e personalize                 6
           requested                   5
           search_results             10
           show                       14
           similar_listings            9

[1732721 rows x 1 columns]

In [197]:
session2

action_type
user_id    action_type                  
00023iyk9l booking_request             1
           click                       4
           data                        9
           message_post                1
           partner_callback            1
...                                  ...
zzzlylp57e click                      22
           data                       17
           message_post                1
           submit                      3
           view                       20

[671364 rows x 1 columns]

In [198]:
session3

action_detail
user_id    action_detail                             
00023iyk9l change_trip_characteristics              2
           confirm_email_link                       1
           dashboard                                4
           header_userpic                           2
           message_post                             1
...                                               ...
zzzlylp57e pending                                  1
           post_checkout_action                     1
           similar_listings                         9
           view_search_results                     12
           wishlist_content_update                  6

[1508994 rows x 1 columns]

In [199]:
session1 = session1.rename(columns={'action':'action_count'}).reset_index()
session2 = session2.rename(columns={'action_type':'action_type_count'}).reset_index()
session3 = session3.rename(columns={'action_detail':'action_detail_count'}).reset_index()

In [200]:
session1

,user_id,action,action_count
0,00023iyk9l,ajax_refresh_subtotal,2
1,00023iyk9l,callback,1
2,00023iyk9l,confirm_email,1
3,00023iyk9l,dashboard,4
4,00023iyk9l,header_userpic,2
...,...,...,...
1732716,zzzlylp57e,personalize,6
1732717,zzzlylp57e,requested,5
1732718,zzzlylp57e,search_results,10
1732719,zzzlylp57e,show,14


In [202]:
session1 = session1.pivot_table(index='user_id', columns='action', values='action_count').fillna(0)

In [203]:
session1

action,10,11,12,15,about_us,accept_decline,account,acculynk_bin_check_failed,acculynk_bin_check_success,acculynk_load_pin_pad,acculynk_pin_pad_error,acculynk_pin_pad_inactive,acculynk_pin_pad_success,acculynk_session_obtained,active,add_business_address_colorbox,add_guest_colorbox,add_guests,add_note,agree_terms_check,agree_terms_uncheck,airbnb_picks,airbrb,ajax_check_dates,ajax_get_referrals_amt,ajax_get_results,ajax_google_translate,ajax_google_translate_description,ajax_google_translate_reviews,ajax_image_upload,ajax_ldp,ajax_lwlb_contact,ajax_payout_edit,ajax_payout_options_by_country,ajax_payout_split_edit,ajax_photo_widget,ajax_photo_widget_form_iframe,ajax_price_and_availability,ajax_referral_banner_experiment_type,ajax_referral_banner_type,ajax_refresh_subtotal,ajax_send_message,ajax_special_offer_dates_available,ajax_statsd,ajax_worth,apply,apply_code,apply_coupon_click,apply_coupon_click_success,apply_coupon_error,apply_coupon_error_type,apply_reservation,approve,ask_question,at_checkpoint,authenticate,authorize,available,badge,become_user,book,booking,braintree_client_token,business_travel,calendar_tab_inner2,callback,campaigns,cancel,cancellation_policies,cancellation_policy_click,change,change_availability,change_currency,change_default_payout,change_password,check,city_count,clear_reservation,click,clickthrough,collections,complete,complete_redirect,complete_status,concierge,confirm_email,confirmation,connect,contact_new,countries,country_options,coupon_code_click,coupon_field_focus,create,create_ach,create_airbnb,create_multiple,create_paypal,currencies,custom_recommended_destinations,dashboard,deactivate,deactivated,deauthorize,decision_tree,delete,department,departments,desks,destroy,detect_fb_session,disaster_action,domains,edit,edit_verification,email_by_key,email_itinerary_colorbox,email_share,email_wishlist,endpoint_error,envoy_bank_details_redirect,envoy_form,events,facebook_auto_login,faq,faq_category,faq_experiment_ids,feed,forgot_password,founders,friend_listing,friends,friends_new,glob,google_importer,guarantee,guest_billing_receipt,guest_booked_elsewhere,handle_vanity_url,hard_fallback_submit,has_profile_pic,header_userpic,home_safety_landing,home_safety_terms,hospitality,hospitality_standards,host_2013,host_cancel,host_summary,hosting_social_proof,how_it_works,identity,image_order,impressions,index,invalid_action,issue,itinerary,jumio,jumio_redirect,jumio_token,kba,kba_update,languages_multiselect,life,listing,listings,load_more,locale_from_host,localization_settings,localized,locations,login,login_modal,lookup,manage_listing,maybe_information,media_resources,message,message_to_host_change,message_to_host_focus,mobile_landing_page,mobile_oauth_callback,multi,multi_message,multi_message_attributes,my,my_listings,my_reservations,new,new_host,new_session,notifications,nyan,office_location,onenight,open_graph_setting,open_hard_fallback_modal,other_hosting_reviews,other_hosting_reviews_first,overview,p4_refund_policy_terms,p4_terms,patch,pay,payment_instruments,payment_methods,payoneer_account_redirect,payoneer_signup_complete,payout_delete,payout_preferences,payout_update,pending,pending_tickets,personalize,phone_number_widget,phone_verification,phone_verification_call_taking_too_long,phone_verification_error,phone_verification_modal,phone_verification_number_submitted_for_call,phone_verification_number_submitted_for_sms,phone_verification_number_sucessfully_submitted,phone_verification_phone_number_removed,phone_verification_success,photography,photography_update,place_worth,plaxo_cb,popular,popular_listing,populate_from_facebook,populate_help_dropdown,position,preapproval,press_content,press_news,press_release,pricing,print_confirmation,privacy,profile_pic,push_notification_callback,qt2,qt_reply_v2,qt_with,questions,rate,reactivate,read_policy_click,receipt,recent_reservations,recommend,recommendation_page,recommendations,recommended_listings,redirect,references,referrer_status,refund_guest_cancellati